In [1]:
import requests
import json
import re
import numpy as np
import pandas as pd

In [29]:
def get_year(year):
    meet = requests.get(year).text
    meet = meet.split("Event ")
    return meet

sex = "Men "
course = "Yard "

yr2007 = get_year("http://meetresults.com/2007/eisl/results.html")
yr2008 = get_year("http://meetresults.com/2008/eisl/results.html")
with open('yr2009.rtf', 'r') as myfile:
    yr2009 = myfile.read()
    yr2009 = yr2009.split("Event ")
yr2010 = get_year("http://www.pennathletics.com//pdf8/731145.pdf")
yr2011 = get_year("http://meetresults.com/2011/ivies/results.html")
yr2012 = get_year("http://www.goprincetontigers.com/ViewArticle.dbml?DB_OEM_ID=10600&ATCLID=205391399")
yr2013 = get_year("http://www.ivyleaguesports.com/championships/mswimdive/2012-13/2013_Ivy_Complete_Meet.htm")
yr2014 = get_year("http://www.ivyleaguesports.com/championships/mswimdive/2013-14/results-splits.html")
yr2015 = get_year("http://www.ivyleaguesports.com/championships/mswimdive/2014-15/results-final.htm")
# yr2016 = get_year("")

In [27]:
def separate_events(year):
    new_list = []
    for event in year:
        if "Time Trial" not in event:
            if "Relay" not in event:
                if "Swim-off" not in event:
                    if "Points" in event:
                        event = event.replace("\\", "")
                        new_list.append(event)
    return new_list

events = separate_events(yr2007)
print(type(events))
print(len(events))
for event in events:
    print(event.split("\n")[0])
    
print(events[6][:200])

<class 'list'>
16
2  Men 500 Yard Freestyle</b>
3  Men 200 Yard IM</b>
4  Men 50 Yard Freestyle</b>
5  Men 1 mtr Diving</b>
8  Men 1000 Yard Freestyle</b>
9  Men 400 Yard IM</b>
10  Men 100 Yard Butterfly</b>
11  Men 200 Yard Freestyle</b>
12  Men 100 Yard Breaststroke</b>
13  Men 100 Yard Backstroke</b>
15  Men 1650 Yard Freestyle</b>
16  Men 200 Yard Backstroke</b>
17  Men 100 Yard Freestyle</b>
18  Men 200 Yard Breaststroke</b>
19  Men 200 Yard Butterfly</b>
20  Men 3 mtr Diving</b>
10  Men 100 Yard Butterfly</b>
 Meet Record: M 47.10        2003 Jesse Gage (Princeton)                       
 EISL 


In [4]:
def split_event_by_line(string):
    if "Points" in string:
        string = string.split("Points")[1]
        string = string.replace("\r","")
        if "Bradley\n" in string:
            string = string.replace("Bradley\n", "Bradley")
        list_of_strings = string.split("\n")
        return list_of_strings


lol = split_event_by_line(events[4])
print(type(lol))
print(len(lol))
print(lol[3])
print("")
lol

<class 'list'>
31
  2 Griest, Robert            SO Princeton                      9:06.64N  28  



[' ',
 '===============================================================================',
 '  1 Wollner, Samuel           JR Harvard                        9:06.36N  32  ',
 '  2 Griest, Robert            SO Princeton                      9:06.64N  28  ',
 '  3 Ashley, Dave              SR Princeton                      9:08.61N  27  ',
 '  4 Lynch, Eric               SO Harvard                        9:10.18N  26  ',
 '  5 Fog, Henning              JR Columbia                       9:12.40N  25  ',
 '  6 Volosin, Peter            JR Brown                          9:15.52N  24  ',
 '  7 Meyer, Alex               FR Harvard                        9:17.08N  23  ',
 '  8 Brunnick, Mason           FR Harvard                        9:17.20N  22  ',
 '  9 Buncher, Stan             JR Princeton                      9:19.88N  20  ',
 ' 10 Ellis, Kevin              SO Dartmouth                      9:21.07N  17  ',
 ' 11 Abbott, Jake              SR Columbia                       9:21.48N  15.5

In [5]:
line = " 12 O'Mara, Michael           SR Brown                1:52.89    1:52.63   15  \r"
print(type(line))

def extract_rank_from_line(line):
    try:
        return int(line[:4])
    except ValueError:
        return 0

def extract_name_from_line(line):
    comma = re.search(r'\,', line).start()
    last_name = line[comma::-1].split(" ")[0][::-1]
    first_name = line[comma+2:].split(" ")[0]
    name = last_name + " " + first_name
    return name

def extract_year_from_line(line):
    years = [" FR ", " SO ", " JR ", " SR "]
    for year in years:
        if year in line:
            return year.replace(" ", "")
        if not any(yr in line for yr in years):
            ix = re.search(r'\d', line[6:]).start()
            return line[ix+6:ix+8]

def extract_school_from_line(line):
    year = extract_year_from_line(line) + " "
    school = line.split(year)[1]
    school = school.split(" ")[0]
    return school

def extract_times_from_line(line):
    time1 = re.compile(r'\d*\:\d\d\.\d\d')
    time2 = re.compile(r'\d\d\d\.\d\d')
    time3 = re.compile(r'\d\d\.\d\d')
    line_ix = [(m.start(0), m.end(0)) for m in re.finditer(time1, line)]
    if len(line_ix) == 0:
        line_ix = [(m.start(0), m.end(0)) for m in re.finditer(time2, line)]
        if len(line_ix) == 0:
            line_ix = [(m.start(0), m.end(0)) for m in re.finditer(time3, line)]        
    if len(line_ix) == 2:
        p_start = line_ix[0][0]
        p_end = line_ix[0][1]
        f_start = line_ix[1][0]
        f_end = line_ix[1][1]
        prelims = line[p_start:p_end]
        finals = line[f_start:f_end]
    elif len(line_ix) == 1:
        f_start = line_ix[0][0]
        f_end = line_ix[0][1]
        finals = line[f_start:f_end]
        prelims = "N/A"
    return [prelims, finals]

def extract_points_from_line(line):
    try: 
        return float(line[-5:])
    except ValueError:
        return 0

def extract_data_from_line(line):
    rank = extract_rank_from_line(line)
    if rank:
        name = extract_name_from_line(line)
        year = extract_year_from_line(line)
        school = extract_school_from_line(line)
        prelim = extract_times_from_line(line)[0]
        final = extract_times_from_line(line)[1]
        points = extract_points_from_line(line)
        return [rank, name, year, school, prelim, final, points]

def extract_all_data_from_event(event):
    list_of_lines = split_event_by_line(event)
    nlist = []
    for line in list_of_lines:
        if line:
            d = extract_data_from_line(line)
            if d:
                nlist.append(d)
    return nlist

extracted_event = extract_all_data_from_event(events[4])
extracted_event

<class 'str'>


[[1, 'Wollner, Samuel', 'JR', 'Harvard', 'N/A', '9:06.36', 32.0],
 [2, 'Griest, Robert', 'SO', 'Princeton', 'N/A', '9:06.64', 28.0],
 [3, 'Ashley, Dave', 'SR', 'Princeton', 'N/A', '9:08.61', 27.0],
 [4, 'Lynch, Eric', 'SO', 'Harvard', 'N/A', '9:10.18', 26.0],
 [5, 'Fog, Henning', 'JR', 'Columbia', 'N/A', '9:12.40', 25.0],
 [6, 'Volosin, Peter', 'JR', 'Brown', 'N/A', '9:15.52', 24.0],
 [7, 'Meyer, Alex', 'FR', 'Harvard', 'N/A', '9:17.08', 23.0],
 [8, 'Brunnick, Mason', 'FR', 'Harvard', 'N/A', '9:17.20', 22.0],
 [9, 'Buncher, Stan', 'JR', 'Princeton', 'N/A', '9:19.88', 20.0],
 [10, 'Ellis, Kevin', 'SO', 'Dartmouth', 'N/A', '9:21.07', 17.0],
 [11, 'Abbott, Jake', 'SR', 'Columbia', 'N/A', '9:21.48', 15.5],
 [11, 'Fee, James', 'FR', 'Pennsylvania', 'N/A', '9:21.48', 15.5],
 [13, 'Reeb, Cody', 'FR', 'Navy', 'N/A', '9:26.39', 14.0],
 [14, 'Gray, Thomas', 'FR', 'Harvard', 'N/A', '9:27.76', 13.0],
 [15, 'Cochran, Nick', 'SO', 'Cornell', 'N/A', '9:28.07', 12.0],
 [16, 'Garber, Kent', 'SR', 'Yale

In [6]:
def get_event_name_num(event):
    if "Diving" not in event:
        try:
            event_num = int(event[:2])
        except ValueError:
            event_num = "NaN"
        dist = int(event.split(sex)[1].split(" ")[0])
        stroke = event.split(course)[1].split("<")[0]
        if (type(event_num) == int) and event_num < 25:
            return event_num, dist, stroke
    else:
        try:
            event_num = int(event[:2])
        except ValueError:
            event_num = "NaN"
        dist = int(event.split(sex)[1].split(" ")[0])
        stroke = "Diving"
        if (type(event_num) == int) and event_num < 25:
            return event_num, dist, stroke

        
for event in events:
    print(get_event_name_num(event))

(2, 500, 'Freestyle')
(3, 200, 'IM')
(4, 50, 'Freestyle')
(5, 1, 'Diving')
(8, 1000, 'Freestyle')
(9, 400, 'IM')
(10, 100, 'Butterfly')
(11, 200, 'Freestyle')
(12, 100, 'Breaststroke')
(13, 100, 'Backstroke')
(15, 1650, 'Freestyle')
(16, 200, 'Backstroke')
(17, 100, 'Freestyle')
(18, 200, 'Breaststroke')
(19, 200, 'Butterfly')
(20, 3, 'Diving')


In [7]:
def pandize_meet(year):
    meet_results = pd.DataFrame(columns = ["Ev#", "Distance", "Stroke", "Rank", "Name", "Yr", "School", "Prelim", "Final", "Points"])

    events_list = separate_events(year)    
    for event in events_list:
        event_name_num = get_event_name_num(event)
        print(event_name_num)
        event_data = extract_all_data_from_event(event)
        pandized_event = pd.DataFrame(event_data)
        pandized_event.columns = ["Rank", "Name", "Yr", "School", "Prelim", "Final", "Points"]
        pandized_event["Ev#"] = event_name_num[0]
        pandized_event["Distance"] = event_name_num[1]
        pandized_event["Stroke"] = event_name_num[2]
        meet_results = meet_results.append(pandized_event)
    return meet_results

pandize_meet(yr2007)

(2, 500, 'Freestyle')
(3, 200, 'IM')
(4, 50, 'Freestyle')
(5, 1, 'Diving')
(8, 1000, 'Freestyle')
(9, 400, 'IM')
(10, 100, 'Butterfly')
(11, 200, 'Freestyle')
(12, 100, 'Breaststroke')
(13, 100, 'Backstroke')
(15, 1650, 'Freestyle')
(16, 200, 'Backstroke')
(17, 100, 'Freestyle')
(18, 200, 'Breaststroke')
(19, 200, 'Butterfly')
(20, 3, 'Diving')


,Distance,Ev#,Final,Name,Points,Prelim,Rank,School,Stroke,Yr
0,500,2,4:24.07,"Smit, Mike",30.0,4:25.94,1,Cornell,Freestyle,SR
1,500,2,4:24.07,"Newman, Wes",30.0,4:26.28,1,Cornell,Freestyle,SO
2,500,2,4:24.19,"Wollner, Samuel",27.0,4:26.38,3,Harvard,Freestyle,JR
3,500,2,4:26.09,"Lynch, Eric",26.0,4:26.51,4,Harvard,Freestyle,SO
4,500,2,4:27.74,"Lee, Hyun",25.0,4:26.12,5,Columbia,Freestyle,SO
5,500,2,4:28.00,"Griest, Robert",24.0,4:28.10,6,Princeton,Freestyle,SO
6,500,2,4:28.74,"Foss, Andrew",23.0,4:27.78,7,Yale,Freestyle,SR
7,500,2,4:32.30,"Ashley, Dave",22.0,4:28.28,8,Princeton,Freestyle,SR
8,500,2,4:27.88,"Brunnick, Mason",20.0,4:29.86,9,Harvard,Freestyle,FR
9,500,2,4:29.19,"Ellis, Kevin",17.0,4:29.18,10,Dartmouth,Freestyle,SO


In [ ]:
pandize_meet(yr2008)

In [30]:
pandize_meet(yr2009)

(2, 500, 'Freestyle========================================================================= \nMeet Record: M 4:20.11 2003 John Cole (Harvard) \nEISL/Ivy Rec: E 4:16.91 2002 John Cole (Harvard) \nPOOL: P 4:20.93 2009 Travis McNamara (Princeton) \nNCAA A: A 4:15.68 \nNCAA B: B 4:24.62 \nName Year School Prelims Finals Points \n========================================================================= \n=== A - Final === \n1 McNamara, Travis FR Princeton 4:20.93 4:18.04M 32 \n23.41 48.94 (25.53) \n1:15.08 (26.14) 1:41.40 (26.32) \n2:07.96 (26.56) 2:34.66 (26.70) \n3:01.25 (26.59) 3:27.89 (26.64) \n3:53.65 (25.76) 4:18.04 (24.39) \n2 Newman, Wes SR Cornell 4:21.15 4:19.26M 28 \n23.70 49.31 (25.61) \n1:15.45 (26.14) 1:41.82 (26.37) \n2:08.06 (26.24) 2:34.65 (26.59) \n3:01.33 (26.68) 3:28.08 (26.75) \n3:54.24 (26.16) 4:19.26 (25.02) \n3 Lynch, Eric SR Harvard 4:21.27 4:19.91M 27 \n24.10 49.93 (25.83) \n1:16.14 (26.21) 1:42.63 (26.49) \n2:09.29 (26.66) 2:35.66 (26.37) \n3:01.94 (26.28) 3:28.1

ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements

In [ ]:
pandize_meet(yr2011).to_csv("IvyData2011.csv")

In [ ]:
pandize_meet(yr2012).to_csv("IvyData2012.csv")

In [ ]:
pandize_meet(yr2013).to_csv("IvyData2013.csv")

In [ ]:
pandize_meet(yr2014).to_csv("IvyData2014.csv")

In [ ]:
pandize_meet(yr2015).to_csv("IvyData2015.csv")

In [12]:
with open('yr2009.rtf', 'r') as myfile:
    yr2009=myfile.read()

In [15]:
yr2009 = yr2009.split("Event ")

In [18]:
len(yr2009)

43

In [21]:
yr2009 = separate_events(yr2009)

In [22]:
extract_all_data_from_event(yr2009[2])

[]

In [25]:
yr2009[2]

'4 Men 50 Yard Freestyle========================================================================= \\\nMeet Record: M 19.49 2009 Doug Lennox (Princeton) \\\nEISL/Ivy Rec: E 19.49 2009 Doug Lennox (Princeton) \\\nPOOL: P 19.49 2009 Doug Lennox (Princeton) \\\nNCAA A: A 19.35 \\\nNCAA B: B 20.02 \\\nName Year School Prelims Finals Points \\\n========================================================================= \\\n=== A - Final === \\\n1 Righi, Alex 09 Yale 19.69 18.90A 32 \\\n2 Lennox, Douglas SR Princeton 19.49 19.36M 28 \\\n3 Ricketts, Dan SR Brown 19.93 19.77B 27 \\\n4 Carter, Michael SR Princeton 19.82 19.91B 26 \\\n5 Guernsey, David SR Harvard 20.11 20.04 25 \\\n6 Powell, Adam SO Columbia 19.84 20.08 24 \\\n7 Baity, Phil SR Cornell 20.17 20.47 23 \\\n8 Monovoukas, Michael FR Princeton 20.19 20.50 22 \\\n=== B - Final === \\\n9 Carey, Kevin SR Cornell 20.59 20.34 20 \\\n10 Pool, Chris 09 Yale 20.20 20.37 17 \\\n11 Wang, Jackson SR Cornell 20.24 20.40 16 \\\n12 Cordes, Colin FR Pr

In [26]:
split_event_by_line(yr2009[2])

[' \\',
 '========================================================================= \\',
 '=== A - Final === \\',
 '1 Righi, Alex 09 Yale 19.69 18.90A 32 \\',
 '2 Lennox, Douglas SR Princeton 19.49 19.36M 28 \\',
 '3 Ricketts, Dan SR Brown 19.93 19.77B 27 \\',
 '4 Carter, Michael SR Princeton 19.82 19.91B 26 \\',
 '5 Guernsey, David SR Harvard 20.11 20.04 25 \\',
 '6 Powell, Adam SO Columbia 19.84 20.08 24 \\',
 '7 Baity, Phil SR Cornell 20.17 20.47 23 \\',
 '8 Monovoukas, Michael FR Princeton 20.19 20.50 22 \\',
 '=== B - Final === \\',
 '9 Carey, Kevin SR Cornell 20.59 20.34 20 \\',
 '10 Pool, Chris 09 Yale 20.20 20.37 17 \\',
 '11 Wang, Jackson SR Cornell 20.24 20.40 16 \\',
 '12 Cordes, Colin FR Princeton 20.29 20.47 15 \\',
 '13 Jones, Bill SR Harvard 20.35 20.50 14 \\',
 '14 Dobrolioubov, Sasha JR Cornell 20.27 20.58 13 \\',
 '15 Bicer, Goksu 12 Yale 20.57 20.61 12 \\',
 '16 Ko, Steve SR Columbia 20.58 20.66 11 \\',
 '=== C - Final === \\',
 '17 Amos, Peter FR Pennsylvania 20.77 